#Troisième partie : modélisation

In [2]:
# on réimporte les librairies
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd # on s'en sert juste pour un affichage un peu plus propre de certains résultats (coefficients des régressions et erreur par pays à la fin)

In [3]:
# on réimporte le dataframe sauvegardé enfin de partie 2 au format "parquet"
df = spark.read.load("/FileStore/parquet/airbnb_part_2")
display(df.limit(50))

id,name,experiences_offered,host_id,host_name,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,jurisdiction_names,cancellation_policy,calculated_host_listings_count,reviews_per_month,geolocation,features,description_len,nb_desc,is_location_exact,host_identity_verified,instant_bookable,host_is_superhost,cancel_strict,cancel_moderate,real_bed,entire_room,apartment,house,hostel,tv,cable_tv,free_parking,pool,gym,air_conditioning,elevator,detectors,appliances,pets,disable,kids,smoking,smart_clothes,fireplace,easy_check_in,breakfast,tub,events,privacy,is_depot,experiences,total_price,price_per_person,market_index,country_index,host_response_time_index,market_dummies,country_dummies,host_response_time_dummies
10015594,Large doubleroom + use of apartment,none,51333380,Kim Brandstrup,GB,within an hour,100,94%,Bruntsfield,1,1,"email,phone,reviews,jumio","Viewforth, Edinburgh, EH10 4LA, United Kingdom",Bruntsfield,Bruntsfield,null,Edinburgh,null,EH10 4LA,Edinburgh,"Edinburgh, United Kingdom",GB,United Kingdom,55.938427989997905,-3.208806747955668,Bed & Breakfast,Private room,2.0,1.0,1,1.0,Real Bed,"tv,wireless internet,kitchen,breakfast,heating,washer,shampoo,hangers,hair dryer,iron,laptop friendly workspace",null,40,null,null,null,0.0,2,10.0,1,1125,6 weeks ago,null,5,10.0,10.0,198.0,43,2015-12-22 00:00:00,2016-07-07 00:00:00,98.0,10.0,10,10,10.0,10.0,10.0,null,null,flexible,1,6.42,"55.93842799,-3.20880674796","Host Is Superhost,Host Has Profile Pic,Host Identity Verified,Is Location Exact",188,7,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,40,20,23.0,1.0,0.0,"List(0, 51, List(23), List(1.0))","List(0, 20, List(1), List(1.0))","List(0, 4, List(0), List(1.0))"
10027322,Termini Smart Room 1,none,51399164,Alessandro,"Rome, Lazio, Italy",within an hour,100,null,Castro Pretorio,10,10,"email,phone,reviews,jumio,government_id","Roma, Lazio 00185, Italy",null,I Centro Storico,null,Roma,Lazio,00185,Rome,"Roma, Italy",IT,Italy,41.90380959757479,12.49960123535744,Bed & Breakfast,Private room,1.0,1.0,1,1.0,Real Bed,"tv,internet,wireless internet,air conditioning,elevator in building,heating,family/kid friendly,first aid kit,safety card,fire extinguisher,essentials,shampoo,lock on bedroom door,hangers,hair dryer,iron,laptop friendly workspace,translation missing: en.hosting_amenity_50,private entrance",null,20,null,null,null,0.0,1,0.0,1,20,today,null,2,18.0,37.0,101.0,156,2015-12-28 00:00:00,2017-05-01 00:00:00,95.0,10.0,10,10,10.0,10.0,10.0,null,null,moderate_new,10,9.4,"41.9038095976,12.4996012354","Host Has Profile Pic,Host Identity Verified,Instant Bookable",182,7,0,1,1,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,1,1,0,0,1,0,1,0,0,0,0,0,1,0,0,20,20,4.0,5.0,0.0,"List(0, 51, List(4), List(1.0))","List(0, 20, List(5), List(1.0))","List(0, 4, List(0), List(1.0))"
10034834,Sailboat Charter in Barcelona BA45,none,8848709,Adela,"Barcelona, Catalonia, Spain",within an hour,90,null,La Vila Olímpica,9,9,"email,phone,reviews,jumio","Barcelona, Catalunya 08005, Spain",null,la Vila Olímpica del Poblenou,Sant Martí,Barcelona,Catalunya,08005,Barcelona,"Barcelona, Spain",ES,Spain,41.387952831093614,2.1983646991790957,Boat,Entire home/apt,10.0,3.5,4,10.0,Real Bed,"kitchen,pets allow

On regroupe notre liste d'indicatrices créées et on ajoute les variables numériques que l'on peut utiliser

In [5]:
liste_var = ["description_len", "nb_desc", "apartment", "house", "hostel", "entire_room", "real_bed", "cancel_strict", "cancel_moderate", "is_location_exact", "host_identity_verified", "instant_bookable", "host_is_superhost", "host_response_rate", "bathrooms", "accommodates", "bedrooms", "price_per_person", "extra_people", "minimum_nights", "availability_365", "number_of_reviews", "review_scores_rating", "reviews_per_month", "tv", "cable_tv", "free_parking", "pool", "gym", "air_conditioning", "elevator", "detectors", "appliances", "pets", "disable", "kids", "smoking", "smart_clothes", "fireplace", "easy_check_in", "breakfast", "tub", "events", "privacy", "is_depot", "experiences", "country_dummies", "host_response_time_dummies"]

df_num = df.select(liste_var)

df_num.printSchema()

root
-- description_len: integer (nullable = true)
-- nb_desc: integer (nullable = true)
-- apartment: integer (nullable = true)
-- house: integer (nullable = true)
-- hostel: integer (nullable = true)
-- entire_room: integer (nullable = true)
-- real_bed: integer (nullable = true)
-- cancel_strict: integer (nullable = true)
-- cancel_moderate: integer (nullable = true)
-- is_location_exact: integer (nullable = true)
-- host_identity_verified: integer (nullable = true)
-- instant_bookable: integer (nullable = true)
-- host_is_superhost: integer (nullable = true)
-- host_response_rate: integer (nullable = true)
-- bathrooms: double (nullable = true)
-- accommodates: double (nullable = true)
-- bedrooms: long (nullable = true)
-- price_per_person: integer (nullable = true)
-- extra_people: double (nullable = true)
-- minimum_nights: integer (nullable = true)
-- availability_365: double (nullable = true)
-- number_of_reviews: integer (nullable = true)
-- review_scores_rating: double (nullable = true)
-- reviews_per_month: string (nullable = true)
-- tv: integer (nullable = true)
-- cable_tv: integer (nullable = true)
-- free_parking: integer (nullable = true)
-- pool: integer (nullable = true)
-- gym: integer (nullable = true)
-- air_conditioning: integer (nullable = true)
-- elevator: integer (nullable = true)
-- detectors: integer (nullable = true)
-- appliances: integer (nullable = true)
-- pets: integer (nullable = true)
-- disable: integer (nullable = true)
-- kids: integer (nullable = true)
-- smoking: integer (nullable = true)
-- smart_clothes: integer (nullable = true)
-- fireplace: integer (nullable = true)
-- easy_check_in: integer (nullable = true)
-- breakfast: integer (nullable = true)
-- tub: integer (nullable = true)
-- events: integer (nullable = true)
-- privacy: integer (nullable = true)
-- is_depot: integer (nullable = true)
-- experiences: integer (nullable = true)
-- country_dummies: vector (nullable = true)
-- host_response_time_dummies: vector (nullable = true)

In [6]:
# il reste une variable caractère : reviews_per_month alors on change ça vite fait
df_num = df_num.withColumn("reviews_per_month", df_num["reviews_per_month"].cast(IntegerType()))

On vérifie s'il reste des valeurs nulles. Si oui, on pourra les remplacer par 0 même si c'est un traitement bif bof à éviter...

In [8]:
#nulls = df_num.select([count(when(col(colname).isNull(), colname)).alias(colname) for colname in df_num.columns]).toPandas()
#nulls
# output : génial nobody.isNull

## Modèle pour expliquer les notes  
L'idée est de voir quelle est l'influence des caractéristiques du logement sur les notes reçues par l'hôte.

In [10]:
# On regroupe toutes les variables numériques avec VectorAssembler
from pyspark.ml.feature import VectorAssembler
name_num = df_num.drop("review_scores_rating").columns
vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
# ! Attention, on a déjà une variable qui s'appelle features !

df_reg = vectorAssembler.transform(df_num)

In [11]:
# On fait tourner le modèle de régression linéaire et on sort les coefficients
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'var_exp', labelCol='review_scores_rating', maxIter=100)
lr_model = lr.fit(df_reg)

Afin de récupérer les coefficients, il y a un petit boulot à faire car les dummy variables construites avec OneHotEncoderEstimator n'ont pas de noms de variable associés. Pour rappel lorsqu'on utilise le one hot encoding de pyspark, on a une variable, par exemple `country_dummies` qui est un sparse vector contenant toutes les dummies

Par conséquent, on a dans `name_num` une 40aine de variable alors qu'on a dans `lr_model.coefficients` une 60aine de valeurs...ce qui est normal puisque certaines variables (les dummies) en contiennent en fait plusieurs. Il faut donc pouvoir faire correspondre chaque coefficient à la bonne variable (et notamment pour ces dummies).

In [13]:
### 1ère étape : construction de la liste des noms de variable en incluant les dummies
# pour ce la on peut passer par les métadonnées du modèle de régression

# application du modèle linéaire sur le df pour obtenir les prédictions
pred = lr_model.transform(df_reg)

# récupération des métadonnées
meta = [f.metadata for f in pred.schema.fields if f.name == 'var_exp'][0]

# récupération des index et des noms des variables du modèle
features_name_index = meta['ml_attr']['attrs']['numeric'] + meta['ml_attr']['attrs']['binary']

# création de la liste des noms de variables du modèle
variables = [item['name'] for item in features_name_index]

#### 2ème étape : les coefficients et les variables associées sous forme de dataframe
liste_coefs = list(lr_model.coefficients)
liste_coefs.append(lr_model.intercept)
variables.append('intercept')
coefs = pd.DataFrame(data=liste_coefs, index=variables, columns=["coefficients"])
coefs.sort_values("coefficients")

,coefficients
country_dummies_Hong Kong,-6.061728
host_response_time_dummies_not available,-3.634457
host_response_time_dummies_within a few hours,-2.785528
country_dummies_Vatican City,-2.771718
host_response_time_dummies_within an hour,-2.712579
host_response_time_dummies_within a day,-2.132746
country_dummies_Denmark,-1.743521
instant_bookable,-1.637130
country_dummies_Ireland,-1.374435
cancel_strict,-1.053291


In [14]:
# le R2 et R2 ajusté : 
print("R2:",lr_model.summary.r2)
print("R2 ajusté:",lr_model.summary.r2adj)

R2: 0.08289145972338419
R2 ajusté: 0.08271681914603957

Le pouvoir prédictif du modèle est assez faible (R2 de 8,2 %) mais on constate quelques résultats intéressants : le fait de louer une maison augmente la note reçue par rapport aux autres logements de 1,3. On constate également que plus on a de reviews, plus la note augmente. Le prix a par contre un effet très faible sur la note reçue. Le fait qu'il y ait une télé, des détecteurs de fumée ou un petit déjeuner inclus augmente aussi sensiblement la note en moyenne, alors que le fait de pouvoir fumer la diminue. En revanche, on voit que certaines caractéristiques de l'hôte notamment sur le temps de réponse ont des effets très négatifs sur la note ce qui est assez contre-intuitif...cela semble rejoindre l'idée que le modèle de prédiction de la note n'est pas vraiment adapté puisque la note dépend plus du séjour en lui-même (sur lequel nous n'avons malheureusement pas d'autre information) que de la partie préliminaire qui consiste à sélectionner le logement. Concentrons-nous donc sur le prix et le nombre d'avis mensuel (comme proxy du nombre de visiteurs).

## Modèle pour expliquer le prix

In [17]:
# On regroupe toutes les variables numériques avec VectorAssembler
from pyspark.ml.feature import VectorAssembler
name_num = df_num.drop("price_per_person").columns
vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
# ! Attention, on a déjà une variable qui s'appelle features !

df_reg = vectorAssembler.transform(df_num)

In [18]:
# On fait tourner le modèle de régression linéaire et on sort les coefficients
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'var_exp', labelCol='price_per_person', maxIter=100)
lr_model = lr.fit(df_reg)

In [19]:
# les coefficients

# liste des noms de variables du modèle
pred = lr_model.transform(df_reg)
meta = [f.metadata for f in pred.schema.fields if f.name == 'var_exp'][0]
features_name_index = meta['ml_attr']['attrs']['numeric'] + meta['ml_attr']['attrs']['binary']
variables = [item['name'] for item in features_name_index]

# coefficients et variables associées sous forme de dataframe
liste_coefs = list(lr_model.coefficients)
liste_coefs.append(lr_model.intercept)
variables.append('intercept')
coefs = pd.DataFrame(data=liste_coefs, index=variables, columns=["coefficients"])
coefs.sort_values("coefficients")

,coefficients
country_dummies_Greece,-40.298199
country_dummies_Austria,-29.595467
country_dummies_Belgium,-28.458692
country_dummies_Spain,-28.236356
country_dummies_Germany,-25.440270
country_dummies_Vatican City,-25.039563
country_dummies_France,-24.826210
country_dummies_Italy,-23.593027
country_dummies_United Kingdom,-19.323640
country_dummies_Mexico,-18.711411


In [20]:
# le R2 et R2 ajusté : 
print("R2:",lr_model.summary.r2)
print("R2 ajusté:",lr_model.summary.r2adj)

R2: 0.7246033772466133
R2 ajusté: 0.7245509347896039

Le pouvoir prédictif du modèle est intéressant (R2 de 72,4 %). On en fera donc un modèle prédictif plus loin dans le notebook. On constate que le prix par personne est le plus important pour les appartements, et pas pour les maisons, sans doute parce qu'elles peuvent accueillir plus de monde. On constate un effet positif des notes reçues par l'hôte avec le prix, ce qui peut s'interpréter ainsi : un hôte avec des bonnes notes peut se permettre de tarifer plus cher que celui qui a de mauvaises notes et est contraint de convaincre le client de réserver en baissant son prix. Le fait d'organiser des évènement lors du séjour augmente fortement le prix (de 3,7 $) et le fait qu'il y ait un dépôt de garantie aussi (2,8 $). L'organisation d'expériences par l'hôte augmente toutes choses égales par ailleurs le prix de 0,7 $.

## Modèle pour expliquer le nombre de notes reçues
Le nombre de retours est un indicateur sur l'attractivité d'un logement puisqu'il permet d'avoir une estimation du nombre de demandes reçues par le propriétaire.

In [23]:
# On regroupe toutes les variables numériques avec VectorAssembler
from pyspark.ml.feature import VectorAssembler
name_num = df_num.drop("reviews_per_month").columns
vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
# ! Attention, on a déjà une variable qui s'appelle features !

df_reg = vectorAssembler.transform(df_num)

In [24]:
# On fait tourner le modèle de régression linéaire et on sort les coefficients
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'var_exp', labelCol='reviews_per_month', maxIter=100, regParam=0.0, elasticNetParam=0.0)
lr_model = lr.fit(df_reg)

In [25]:
# les coefficients

# liste des noms de variables du modèle
pred = lr_model.transform(df_reg)
meta = [f.metadata for f in pred.schema.fields if f.name == 'var_exp'][0]
features_name_index = meta['ml_attr']['attrs']['numeric'] + meta['ml_attr']['attrs']['binary']
variables = [item['name'] for item in features_name_index]

# coefficients et variables associées sous forme de dataframe
liste_coefs = list(lr_model.coefficients)
liste_coefs.append(lr_model.intercept)
variables.append('intercept')
coefs = pd.DataFrame(data=liste_coefs, index=variables, columns=["coefficients"])
coefs.sort_values("coefficients")

,coefficients
experiences,-0.585115
country_dummies_Italy,-0.325505
intercept,-0.302058
host_response_time_dummies_not available,-0.277469
country_dummies_Spain,-0.226532
is_location_exact,-0.210572
country_dummies_France,-0.195230
country_dummies_Greece,-0.183231
hostel,-0.181048
country_dummies_Mexico,-0.179818


In [26]:
# le R2 et R2 ajusté : 
print("R2:",lr_model.summary.r2)
print("R2 ajusté:",lr_model.summary.r2adj)

R2: 0.41316156645093627
R2 ajusté: 0.41304981762477

Le pouvoir prédictif du modèle est intéressant (R2 de 41,3 %). On constate que les variables ayant le plus d'effet sur le nombre de notes reçues par mois sont la variable indicatrice indiquant si le logement peut être réservé instantanément (`instant_bookable`) et celle indiquant si l'hôte répond généralement en moins d'1h (`host_response_time_dummies_within an hour`). Ces caractéristiques augmentent le nombre de reviews par mois de respectivement 0,47 et 0,53 en moyenne et donc le nombre de réservations. On constate par contre une absence d'effet du fait d'être un "super-hôte" airbnb.

## Modèle prédictif du prix  
On peut imaginer un modèle visant à aider les propriétaires à ajuster le prix de leur nuitée en fonction des caractéristiques de leur logement.

On sort quelques stats sur les prix pour voir sur quels ordres de grandeur on est :

In [30]:
# On compare aux statistiques sur le prix : 
df_reg.select("price_per_person").describe().show()

+-------+------------------+
summary| price_per_person|
+-------+------------------+
 count| 362417|
 mean|61.611116476324234|
 stddev| 66.15269739544351|
 min| 0|
 max| 1201|
+-------+------------------+

On essaye un première modèle de régression linéaire classique :

In [32]:
# On regroupe toutes les variables numériques avec VectorAssembler
from pyspark.ml.feature import VectorAssembler
name_num = df_num.drop("price_per_person").columns
vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
# ! Attention, on a déjà une variable qui s'appelle features !

df_reg = vectorAssembler.transform(df_num)

# On splite notre base
splits = df_reg.randomSplit([0.7, 0.3])
train_price = splits[0]
test_price = splits[1]

lr = LinearRegression(featuresCol = 'var_exp', labelCol='price_per_person', maxIter=100)
lr_train_test = lr.fit(train_price)

# on applique la prédiction
lr_predictions = lr_train_test.transform(test_price)

# on sort MAE et rmse pour juger de la qualité de la prédiction
test_result = lr_train_test.evaluate(test_price)
# RMSE & MAE
print(test_result.rootMeanSquaredError, test_result.meanAbsoluteError)

35.289088750964716 19.63547355905357

L'erreur moyenne de prédiction est autour de 19,5$.  
On peut faire varier `elasticNetParam`, qui permet de pénaliser le modèle, et voir les effets sur la qualité de la prédiction. On retombe en effet sur un modèle de Lasso quand il est égal à 1 et sur un modèle de Ridge quand il est égal à 0 (voir [la documentation](https://spark.apache.org/docs/1.5.2/ml-linear-methods.html)).

In [34]:
from pyspark.ml.feature import MinMaxScaler
# On fait un scaler avant d'appliquer la régression pénalisée
scaler = MinMaxScaler(inputCol="var_exp", outputCol="var_exp_scale")
scaler_model = scaler.fit(train_price)
train_price_scale = scaler_model.transform(train_price)
test_price_scale = scaler_model.transform(test_price)


lr = LinearRegression(featuresCol = 'var_exp_scale', labelCol='price_per_person', maxIter=100, regParam = 0.1, elasticNetParam = 0)
lr_train_test = lr.fit(train_price_scale)

# on applique la prédiction
lr_predictions = lr_train_test.transform(test_price_scale)

# on sort MAE et rmse pour juger de la qualité de la prédiction
test_result = lr_train_test.evaluate(test_price_scale)
# RMSE & MAE
print(test_result.rootMeanSquaredError, test_result.meanAbsoluteError)

35.2885665589981 19.627394252453932

La pénalisation n'a pas beaucoup d'impact. On peut tester d'autres modèles de régression pour voir si on arrive encore à baisser cette moyenne.

In [36]:
# Random Forest
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor(featuresCol="var_exp", labelCol='price_per_person')

rf_trained = rf.fit(train_price)
# on applique la prédiction
rf_predictions = rf_trained.transform(test_price)  

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="price_per_person", predictionCol="prediction", metricName="mae")

print("Moyenne de l'erreur en valeur absolue (MAE) sur base de test = %g" % evaluator.evaluate(rf_predictions))


Moyenne de l'erreur en valeur absolue (MAE) sur base de test = 22.3719

Le modèle random forest fait moins bien que le modèle linéaire, essayons le gradient boosted trees

In [38]:
# Gradient Boosted Tree
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

gbt = GBTRegressor(featuresCol="var_exp", labelCol='price_per_person', maxIter=50)

gbt_trained = gbt.fit(train_price)
# on applique la prédiction
gbt_predictions = gbt_trained.transform(test_price)  

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="price_per_person", predictionCol="prediction", metricName="mae")

print("Moyenne de l'erreur en valeur absolue (MAE) sur base de test = %g $" % evaluator.evaluate(gbt_predictions))


Moyenne de l'erreur en valeur absolue (MAE) sur base de test = 16.8585 $

C'est le gradient boosted tree qui nous donne les meilleurs résultats en termes de prédictions avec un MAE toujours inférieure aux précédents modèle et qui se situe autour de 16,7$.

## Modèle différencié par pays
Plutôt que de contrôler par `country_dummies`, on peut affiner et faire autant de modèles que de pays en mettant tout ça dans une fonction. Pour tenter d'améliorer un peu les prédictions et conserver une indicatrice géographique dans les modèles par pays, on peut ausi ajouter `market_dummies` qui correspond plus ou moins aux villes (maille géographique un peu plus large que la ville).

In [41]:
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

def pred_by_country(country, df = df):
  liste_var = ["description_len", "nb_desc", "apartment", "house", "hostel", "entire_room", "real_bed", "cancel_strict", "cancel_moderate", "is_location_exact", "host_identity_verified", "instant_bookable",
               "host_is_superhost", "host_response_rate", "bathrooms", "accommodates", "bedrooms", "price_per_person", "extra_people", "minimum_nights", "availability_365", "number_of_reviews",
               "review_scores_rating", "reviews_per_month", "tv", "cable_tv", "free_parking", "pool", "gym", "air_conditioning", "elevator", "detectors", "appliances", "pets", "disable", "kids", 
               "smoking", "smart_clothes", "fireplace", "easy_check_in", "breakfast", "tub", "events", "privacy", "is_depot", "experiences", "host_response_time_dummies", "market_dummies"]
  
  df = df.withColumn("reviews_per_month", df["reviews_per_month"].cast(IntegerType()))
  df_num = df.filter(df.country == country).select(liste_var)
  
  name_num = df_num.drop("price_per_person").columns
  vectorAssembler = VectorAssembler(inputCols = name_num, outputCol = 'var_exp') 
  # ! Attention, on a déjà une variable qui s'appelle features !
  df_reg = vectorAssembler.transform(df_num)
  
  # On splite
  splits = df_reg.randomSplit([0.7, 0.3])
  train_price = splits[0]
  test_price = splits[1]
  
  # On fait le GBT
  gbt = GBTRegressor(featuresCol="var_exp", labelCol='price_per_person', maxIter=50)

  gbt_trained = gbt.fit(train_price)
  # on applique la prédiction
  gbt_predictions = gbt_trained.transform(test_price)  

  # Select (prediction, true label) and compute test error
  evaluator = RegressionEvaluator(
  labelCol="price_per_person", predictionCol="prediction", metricName="mae")

  return(evaluator.evaluate(gbt_predictions))
  

Regardons les différents pays à disposition et les effectifs avant de voir la qualité du modèle dans chacun des pays :

In [43]:
display(df.groupBy("country").count())

country,count
Germany,15888
France,41536
Greece,3887
Belgium,5827
United States,101485
China,17
Italy,24644
Spain,32697
Denmark,13180
Ireland,5285


Affichons les résultats de la prédiction pour certains pays :

In [45]:
res = pd.DataFrame(index=["United States", "United Kingdom", "France", "Spain", "Australia", "Italy", "Canada", "Germany", "Netherlands"],
                   columns=["MAE", "Prix moyen"])

# calcul des moyennes de pays
means = dict(df.groupBy("country").avg("price_per_person").collect())

In [46]:
# On découpe en 3 cellules sinon c'est trop long (plus de 2h...) et le cluster se termine
# calcul des erreurs de prédictions et ajout des moyennes dans le df
for pays in ["United States", "United Kingdom", "France"]:
  res.loc[pays,"MAE"] = pred_by_country(pays)
  res.loc[pays,"Prix moyen"] = means[pays]

In [47]:
# calcul des erreurs de prédictions et ajout des moyennes dans le df
for pays in ["Spain", "Australia", "Italy"]:
  res.loc[pays,"MAE"] = pred_by_country(pays)
  res.loc[pays,"Prix moyen"] = means[pays]

In [48]:
# calcul des erreurs de prédictions et ajout des moyennes dans le df
for pays in ["Canada", "Germany", "Netherlands"]:
  res.loc[pays,"MAE"] = pred_by_country(pays)
  res.loc[pays,"Prix moyen"] = means[pays]

In [49]:
# ajout des erreurs relatives par pays  
res["Erreur absolue relative"] = res["MAE"]/res["Prix moyen"]
res

,MAE,Prix moyen,Erreur absolue relative
United States,18.1821,66.3385,0.27408
United Kingdom,10.1985,39.1737,0.26034
France,10.6389,41.0895,0.258919
Spain,8.79087,32.9147,0.26708
Australia,18.2191,68.8367,0.264671
Italy,8.87494,36.3542,0.244124
Canada,14.1153,52.4991,0.268868
Germany,8.43384,32.4495,0.259907
Netherlands,13.117,61.2844,0.214035


Pour conclure, les performances du modèle varient un peu en fonction des pays avec une erreur absolue relative qui varie entre 21 et 27%.